In [3]:
import pygame
import math

pygame.init()
screen = pygame.display.set_mode((1200, 800), pygame.RESIZABLE)
pygame.display.set_caption("Солнечная система")
WIDTH, HEIGHT = 1200, 800
cx = WIDTH // 2
cy = HEIGHT // 2   
FPS = 60
clock = pygame.time.Clock()


class Planet:
    def __init__(self, screen, radius, orbit_radius, color=None, speed=0, angle=0, image_path=None, name=""):
        self.screen = screen
        self.radius = radius
        self.orbit_radius = orbit_radius
        self.color = color
        self.speed = speed
        self.angle = angle
        self.x = 0
        self.y = 0
        self.name = name
        self.image = None
        if image_path:
            try:
                self.image = pygame.image.load(image_path).convert_alpha()
                self.image = pygame.transform.scale(self.image, (radius*2, radius*2))
            except:
                self.image = None
                print(f"Не удалось загрузить текстуру для {name}")

    def update(self, dt):
        global cx, cy
        self.angle += self.speed * dt
        self.x = cx + self.orbit_radius * math.cos(self.angle)
        self.y = cy + self.orbit_radius * math.sin(self.angle)

    def draw(self):
        # Рисуем орбиту (полупрозрачную)
        if self.orbit_radius > 0:
            orbit_surface = pygame.Surface((self.orbit_radius*2, self.orbit_radius*2), pygame.SRCALPHA)
            pygame.draw.circle(orbit_surface, (100, 100, 100, 50), (self.orbit_radius, self.orbit_radius), self.orbit_radius, 1)
            self.screen.blit(orbit_surface, (cx - self.orbit_radius, cy - self.orbit_radius))
        
        # Рисуем планету
        if self.image:
            self.screen.blit(self.image, (int(self.x - self.radius), int(self.y - self.radius)))
        else:
            pygame.draw.circle(self.screen, self.color, (int(self.x), int(self.y)), self.radius)


class Moon:
    def __init__(self, screen, radius, orbit_radius, planet, color=None, speed=0, angle=0, image_path=None):
        self.screen = screen
        self.radius = radius
        self.orbit_radius = orbit_radius
        self.planet = planet
        self.color = color
        self.speed = speed
        self.angle = angle
        self.x = 0
        self.y = 0
        self.image = None
        if image_path:
            try:
                self.image = pygame.image.load(image_path).convert_alpha()
                self.image = pygame.transform.scale(self.image, (radius*2, radius*2))
            except:
                self.image = None

    def update(self, dt):
        self.angle += self.speed * dt
        # Луна вращается вокруг своей планеты
        self.x = self.planet.x + self.orbit_radius * math.cos(self.angle)
        self.y = self.planet.y + self.orbit_radius * math.sin(self.angle)

    def draw(self):
        if self.image:
            self.screen.blit(self.image, (int(self.x - self.radius), int(self.y - self.radius)))
        else:
            pygame.draw.circle(self.screen, self.color, (int(self.x), int(self.y)), self.radius)


# Создаем Солнце и планеты
sun_color = (255, 200, 0)
sun_radius = 30

# Создаем все 8 планет с приблизительными параметрами
mercury = Planet(screen, radius=8, orbit_radius=60, color=(169, 169, 169), speed=4.7, angle=0, image_path="mercury.png", name="Mercury")
venus = Planet(screen, radius=12, orbit_radius=90, color=(255, 165, 0), speed=3.5, angle=1.5, image_path="venus.png", name="Venus")
earth = Planet(screen, radius=13, orbit_radius=120, color=(0, 100, 200), speed=2.9, angle=3.0, image_path="earth.png", name="Earth")
mars = Planet(screen, radius=10, orbit_radius=150, color=(255, 100, 80), speed=2.4, angle=4.5, image_path="mars.png", name="Mars")
jupiter = Planet(screen, radius=25, orbit_radius=200, color=(255, 215, 0), speed=1.3, angle=6.0, image_path="jupiter.png", name="Jupiter")
saturn = Planet(screen, radius=22, orbit_radius=260, color=(210, 180, 140), speed=0.9, angle=7.5, image_path="saturn.png", name="Saturn")
uranus = Planet(screen, radius=18, orbit_radius=320, color=(173, 216, 230), speed=0.5, angle=9.0, image_path="uranus.png", name="Uranus")
neptune = Planet(screen, radius=17, orbit_radius=380, color=(65, 105, 225), speed=0.4, angle=10.5, image_path="neptune.png", name="Neptune")

# Создаем Луну, которая вращается вокруг Земли
moon = Moon(screen, radius=4, orbit_radius=20, planet=earth, color=(200, 200, 200), speed=5.0, angle=0, image_path="moon.png")

# Список всех планет для удобного обновления и отрисовки
planets = [mercury, venus, earth, mars, jupiter, saturn, uranus, neptune]

running = True
while running:
    dt = clock.tick(FPS) / 1000.0
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.VIDEORESIZE:
            WIDTH, HEIGHT = event.w, event.h
            screen = pygame.display.set_mode((WIDTH, HEIGHT), pygame.RESIZABLE)
            cx = WIDTH // 2
            cy = HEIGHT // 2  

    # Обновляем позиции всех планет
    for planet in planets:
        planet.update(dt)
    
    # Обновляем позицию Луны
    moon.update(dt)
    
    # Отрисовка
    screen.fill((0, 0, 20))  # Темно-синий фон
    
    # Рисуем Солнце
    pygame.draw.circle(screen, sun_color, (cx, cy), sun_radius)
    
    # Рисуем все планеты
    for planet in planets:
        planet.draw()
    
    # Рисуем Луну
    moon.draw()
    
    pygame.display.flip()

pygame.quit()

In [11]:
import pygame
import math
import random

pygame.init()
screen = pygame.display.set_mode((1200, 800), pygame.RESIZABLE)
pygame.display.set_caption("Космическая пушка: Сбей все планеты!")
WIDTH, HEIGHT = 1200, 800
cx = WIDTH // 2
cy = HEIGHT // 2   
FPS = 60
clock = pygame.time.Clock()

# Загрузка звуков
try:
    pygame.mixer.init()
    score_sound = pygame.mixer.Sound("score.wav")
    victory_sound = pygame.mixer.Sound("water-bubbles-4.mp3")
except:
    print("Звуковые файлы не найдены")

class Planet:
    def __init__(self, screen, radius, orbit_radius, color=None, speed=0, angle=0, image_path=None, name=""):
        self.screen = screen
        self.radius = radius
        self.orbit_radius = orbit_radius
        self.color = color
        self.speed = speed
        self.angle = angle
        self.x = 0
        self.y = 0
        self.name = name
        self.image = None
        self.rect = pygame.Rect(0, 0, radius*2, radius*2)
        self.active = True
        if image_path:
            try:
                self.image = pygame.image.load(image_path).convert_alpha()
                self.image = pygame.transform.scale(self.image, (radius*2, radius*2))
            except:
                self.image = None
                print(f"Не удалось загрузить текстуру для {name}")

    def update(self, dt):
        if self.active:
            global cx, cy
            self.angle += self.speed * dt
            self.x = cx + self.orbit_radius * math.cos(self.angle)
            self.y = cy + self.orbit_radius * math.sin(self.angle)
            self.rect.center = (int(self.x), int(self.y))

    def draw(self):
        if self.active:
            # Рисуем орбиту (полупрозрачную)
            if self.orbit_radius > 0:
                orbit_surface = pygame.Surface((self.orbit_radius*2, self.orbit_radius*2), pygame.SRCALPHA)
                pygame.draw.circle(orbit_surface, (100, 100, 100, 50), (self.orbit_radius, self.orbit_radius), self.orbit_radius, 1)
                self.screen.blit(orbit_surface, (cx - self.orbit_radius, cy - self.orbit_radius))
            
            # Рисуем планету как прямоугольник с текстурой или цветом
            if self.image:
                self.screen.blit(self.image, (int(self.x - self.radius), int(self.y - self.radius)))
            else:
                pygame.draw.rect(self.screen, self.color, self.rect)

    def hit(self):
        self.active = False
        return True

class Moon:
    def __init__(self, screen, radius, orbit_radius, planet, color=None, speed=0, angle=0, image_path=None):
        self.screen = screen
        self.radius = radius
        self.orbit_radius = orbit_radius
        self.planet = planet
        self.color = color
        self.speed = speed
        self.angle = angle
        self.x = 0
        self.y = 0
        self.rect = pygame.Rect(0, 0, radius*2, radius*2)
        self.image = None
        if image_path:
            try:
                self.image = pygame.image.load(image_path).convert_alpha()
                self.image = pygame.transform.scale(self.image, (radius*2, radius*2))
            except:
                self.image = None

    def update(self, dt):
        if self.planet.active:
            self.angle += self.speed * dt
            # Луна вращается вокруг своей планеты
            self.x = self.planet.x + self.orbit_radius * math.cos(self.angle)
            self.y = self.planet.y + self.orbit_radius * math.sin(self.angle)
            self.rect.center = (int(self.x), int(self.y))

    def draw(self):
        if self.planet.active:
            if self.image:
                self.screen.blit(self.image, (int(self.x - self.radius), int(self.y - self.radius)))
            else:
                pygame.draw.rect(self.screen, self.color, self.rect)

class Cannon:
    def __init__(self, screen):
        self.screen = screen
        self.x = 50
        self.y = HEIGHT - 50
        self.angle = 45
        self.power = 12
        self.projectiles = []
        self.gravity = 0.5
        self.on_ground = True
        self.radius = 8
        
    def update(self, dt):
        # Обновляем позиции снарядов
        for proj in self.projectiles[:]:
            proj['vy'] += self.gravity
            proj['x'] += proj['vx']
            proj['y'] += proj['vy']
            
            # Проверка выхода за границы экрана
            if (proj['x'] < 0 or proj['x'] > WIDTH or 
                proj['y'] < 0 or proj['y'] > HEIGHT):
                self.projectiles.remove(proj)
                self.on_ground = True
    
    def shoot(self):
        if self.on_ground:
            rad = math.radians(self.angle)
            vx = self.power * math.cos(rad)
            vy = -self.power * math.sin(rad)
            self.projectiles.append({
                'x': self.x, 'y': self.y, 
                'vx': vx, 'vy': vy,
                'radius': self.radius
            })
            self.on_ground = False
    
    def draw(self):
        # Рисуем пушку
        pygame.draw.circle(self.screen, (100, 100, 100), (self.x, self.y), 15)
        
        # Рисуем ствол пушки
        rad = math.radians(self.angle)
        end_x = self.x + 30 * math.cos(rad)
        end_y = self.y - 30 * math.sin(rad)
        pygame.draw.line(self.screen, (50, 50, 50), (self.x, self.y), (end_x, end_y), 8)
        
        # Рисуем траекторию прицеливания
        if self.on_ground:
            points = []
            rad = math.radians(self.angle)
            temp_vx = self.power * math.cos(rad)
            temp_vy = -self.power * math.sin(rad)
            temp_x, temp_y = self.x, self.y
            for i in range(60):
                temp_vy += self.gravity
                temp_x += temp_vx
                temp_y += temp_vy
                if temp_x < 0 or temp_x > WIDTH or temp_y < 0 or temp_y > HEIGHT:
                    break
                points.append((int(temp_x), int(temp_y)))
            if len(points) > 1:
                pygame.draw.lines(self.screen, (255, 255, 255, 100), False, points, 2)
        
        # Рисуем снаряды
        for proj in self.projectiles:
            pygame.draw.circle(self.screen, (255, 0, 0), (int(proj['x']), int(proj['y'])), proj['radius'])

# Создаем Солнце с возможностью текстуры
sun_image = None
try:
    sun_image = pygame.image.load("sun.png").convert_alpha()
    sun_image = pygame.transform.scale(sun_image, (60, 60))
except:
    sun_image = None

sun_color = (255, 200, 0)
sun_radius = 30

# Создаем все 8 планет с приблизительными параметрами
mercury = Planet(screen, radius=8, orbit_radius=60, color=(169, 169, 169), speed=4.1, angle=0, image_path="mercury.png", name="Mercury")
venus = Planet(screen, radius=12, orbit_radius=90, color=(255, 165, 0), speed=3.2, angle=1.5, image_path="venus.png", name="Venus")
earth = Planet(screen, radius=13, orbit_radius=120, color=(0, 100, 200), speed=2.7, angle=3.0, image_path="earth.png", name="Earth")
mars = Planet(screen, radius=10, orbit_radius=150, color=(255, 100, 80), speed=2.3, angle=4.5, image_path="mars.png", name="Mars")
jupiter = Planet(screen, radius=25, orbit_radius=200, color=(255, 215, 0), speed=1.3, angle=6.0, image_path="jupiter.png", name="Jupiter")
saturn = Planet(screen, radius=22, orbit_radius=260, color=(210, 180, 140), speed=0.9, angle=7.5, image_path="saturn.png", name="Saturn")
uranus = Planet(screen, radius=18, orbit_radius=320, color=(173, 216, 230), speed=0.7, angle=9.0, image_path="uranus.png", name="Uranus")
neptune = Planet(screen, radius=17, orbit_radius=380, color=(65, 105, 225), speed=0.5, angle=10.5, image_path="neptune.png", name="Neptune")

# Создаем Луну, которая вращается вокруг Земли
moon = Moon(screen, radius=4, orbit_radius=20, planet=earth, color=(200, 200, 200), speed=5.0, angle=0, image_path="moon.png")

# Создаем пушку
cannon = Cannon(screen)

# Список всех планет для удобного обновления и отрисовки
planets = [mercury, venus, earth, mars, jupiter, saturn, uranus, neptune]
total_planets = len(planets)

# Переменные для счета и состояния игры
score = 0
game_won = False
victory_sound_played = False
font = pygame.font.Font(None, 36)
large_font = pygame.font.Font(None, 72)

running = True
while running:
    dt = clock.tick(FPS) / 1000.0
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.VIDEORESIZE:
            WIDTH, HEIGHT = event.w, event.h
            screen = pygame.display.set_mode((WIDTH, HEIGHT), pygame.RESIZABLE)
            cx = WIDTH // 2
            cy = HEIGHT // 2
        elif event.type == pygame.KEYDOWN:
            if not game_won:  # Только если игра не закончена
                if event.key == pygame.K_UP:
                    cannon.angle = min(cannon.angle + 5, 80)
                elif event.key == pygame.K_DOWN:
                    cannon.angle = max(cannon.angle - 5, 10)
                elif event.key == pygame.K_w:
                    cannon.power = min(cannon.power + 1, 25)
                elif event.key == pygame.K_s:
                    cannon.power = max(cannon.power - 1, 5)
                elif event.key == pygame.K_SPACE:
                    cannon.shoot()

    if not game_won:
        # Обновляем позиции всех планет
        for planet in planets:
            planet.update(dt)
        
        # Обновляем позицию Луны
        moon.update(dt)
        
        # Обновляем пушку и снаряды
        cannon.update(dt)
        
        # Проверяем столкновения снарядов с планетами
        for proj in cannon.projectiles[:]:
            proj_rect = pygame.Rect(proj['x'] - proj['radius'], proj['y'] - proj['radius'], 
                                   proj['radius'] * 2, proj['radius'] * 2)
            
            for planet in planets:
                if planet.active and proj_rect.colliderect(planet.rect):
                    cannon.projectiles.remove(proj)
                    planet.hit()
                    score += 1
                    try:
                        score_sound.play()
                    except:
                        pass
                    cannon.on_ground = True
                    break
        
        # Проверяем победу
        if score >= total_planets:
            game_won = True
            if not victory_sound_played:
                try:
                    victory_sound.play()
                except:
                    pass
                victory_sound_played = True

    # Отрисовка
    screen.fill((0, 0, 20))  # Темно-синий фон
    
    # Рисуем Солнце (с текстурой или цветом)
    if sun_image:
        screen.blit(sun_image, (cx - sun_radius, cy - sun_radius))
    else:
        pygame.draw.circle(screen, sun_color, (cx, cy), sun_radius)
    
    # Рисуем все планеты
    for planet in planets:
        planet.draw()
    
    # Рисуем Луну
    moon.draw()
    
    # Рисуем пушку
    cannon.draw()
    
    # Отображаем счет и параметры пушки
    score_text = font.render(f"Сбито планет: {score}/{total_planets}", True, (255, 255, 255))
    angle_text = font.render(f"Угол: {cannon.angle}°", True, (255, 255, 255))
    power_text = font.render(f"Сила: {cannon.power}", True, (255, 255, 255))
    
    screen.blit(score_text, (10, 10))
    screen.blit(angle_text, (10, 50))
    screen.blit(power_text, (10, 90))
    
    # Инструкция
    if not game_won:
        instr_text = font.render("Управление: Стрелки - угол, W/S - сила, Пробел - выстрел", True, (200, 200, 200))
        screen.blit(instr_text, (10, HEIGHT - 40))
    
    # Показываем сообщение о победе
    if game_won:
        victory_bg = pygame.Surface((WIDTH, 100), pygame.SRCALPHA)
        victory_bg.fill((0, 0, 0, 150))
        screen.blit(victory_bg, (0, HEIGHT//2 - 50))
        
        victory_text = large_font.render("ПОБЕДА!", True, (0, 255, 0))
        victory_rect = victory_text.get_rect(center=(WIDTH//2, HEIGHT//2 - 20))
        screen.blit(victory_text, victory_rect)
        
        restart_text = font.render("Все планеты сбиты! Нажмите R для новой игры", True, (255, 255, 255))
        restart_rect = restart_text.get_rect(center=(WIDTH//2, HEIGHT//2 + 20))
        screen.blit(restart_text, restart_rect)
        
        # Перезапуск игры
        keys = pygame.key.get_pressed()
        if keys[pygame.K_r]:
            # Сброс всех планет
            for planet in planets:
                planet.active = True
            score = 0
            game_won = False
            victory_sound_played = False
            cannon.projectiles.clear()
            cannon.on_ground = True
    
    pygame.display.flip()

pygame.quit()

In [25]:
import pygame
import math
import random

pygame.init()
screen = pygame.display.set_mode((1200, 800), pygame.RESIZABLE)
pygame.display.set_caption("Солнечная система")
WIDTH, HEIGHT = 1200, 800
cx = WIDTH // 2
cy = HEIGHT // 2   
FPS = 60
clock = pygame.time.Clock()

#музыка
pygame.mixer.init()
pygame.mixer.music.load("cosmo_music.mp3")
pygame.mixer.music.play(-1) 

class Planet:
    def __init__(self, screen, radius, orbit_radius, color=None, speed=0, angle=0, image_path=None, name=""):
        self.screen = screen
        self.radius = radius
        self.orbit_radius = orbit_radius
        self.color = color
        self.speed = speed
        self.angle = angle
        self.x = 0
        self.y = 0
        self.name = name
        self.image = None
        self.rect = pygame.Rect(0, 0, radius*2, radius*2)
        self.active = True
        if image_path:
            self.image = pygame.image.load(image_path).convert_alpha()
            self.image = pygame.transform.scale(self.image, (radius*2, radius*2))

    def update(self, dt):
        if self.active:
            global cx, cy
            self.angle += self.speed * dt
            self.x = cx + self.orbit_radius * math.cos(self.angle)
            self.y = cy + self.orbit_radius * math.sin(self.angle)
            self.rect.center = (int(self.x), int(self.y))

    def draw(self):
        if self.active:
            #орбиты планет
            if self.orbit_radius > 0:
                orbit_surface = pygame.Surface((self.orbit_radius*2, self.orbit_radius*2), pygame.SRCALPHA)
                pygame.draw.circle(orbit_surface, (100, 100, 100, 50), (self.orbit_radius, self.orbit_radius), self.orbit_radius, 1)
                self.screen.blit(orbit_surface, (cx - self.orbit_radius, cy - self.orbit_radius))
            
            #отрисовка планет
            if self.image:
                self.screen.blit(self.image, (int(self.x - self.radius), int(self.y - self.radius)))
            else:
                pygame.draw.rect(self.screen, self.color, self.rect)

    def hit(self):
        self.active = False
        return True

class Moon:
    def __init__(self, screen, radius, orbit_radius, planet, color=None, speed=0, angle=0, image_path=None):
        self.screen = screen
        self.radius = radius
        self.orbit_radius = orbit_radius
        self.planet = planet
        self.color = color
        self.speed = speed
        self.angle = angle
        self.x = 0
        self.y = 0
        self.rect = pygame.Rect(0, 0, radius*2, radius*2)
        self.image = None
        if image_path:
            try:
                self.image = pygame.image.load(image_path).convert_alpha()
                self.image = pygame.transform.scale(self.image, (radius*2, radius*2))
            except:
                self.image = None

    def update(self, dt):
        if self.planet.active:
            self.angle += self.speed * dt
            #луна вращается вокруг земли
            self.x = self.planet.x + self.orbit_radius * math.cos(self.angle)
            self.y = self.planet.y + self.orbit_radius * math.sin(self.angle)
            self.rect.center = (int(self.x), int(self.y))

    def draw(self):
        if self.planet.active:
            if self.image:
                self.screen.blit(self.image, (int(self.x - self.radius), int(self.y - self.radius)))
            else:
                pygame.draw.rect(self.screen, self.color, self.rect)

class Comet:
    def __init__(self, screen, image_path=None):
        self.screen = screen
        self.width = 80  
        self.height = 40
        self.rect = pygame.Rect(0, 0, self.width, self.height)
        self.active = False
        self.speed = 5
        self.spawn_timer = 0
        self.spawn_interval = 7000  #7 секунд
        self.image = None
        
        #загрузка изображения кометы
        if image_path:
            try:
                self.image = pygame.image.load(image_path).convert_alpha()
                self.image = pygame.transform.scale(self.image, (self.width, self.height))
            except Exception as e:
                self.image = None
    
    def update(self, dt):
        #обновляем таймер спавна
        self.spawn_timer += dt * 1000
        
        #если пришло время спавна и комета не активна
        if self.spawn_timer >= self.spawn_interval and not self.active:
            self.spawn()
            self.spawn_timer = 0
        
        #если комета активна, двигаем ее
        if self.active:
            self.rect.x += self.speed
            #если комета улетела за левую границу удаляем её
            if self.rect.right < 0:
                self.active = False
    
    def spawn(self):
        #появление кометы
        self.rect.x = 0
        self.rect.y = 50  #фикс высота вверху экрана
        self.active = True

    def draw(self):
        if self.active:
            if self.image:
                self.screen.blit(self.image, self.rect)
            else:
                pygame.draw.rect(self.screen, (150, 150, 150), self.rect)

#параметры для броска шарика
start_x, start_y = 100, 700
x, y = start_x, start_y
ball_radius = 15
vx, vy = 0, 0
gravity = 0.5
power = 12
angle = 45
bounce_loss = 0.7
on_ground = True

#солнце
sun_image = pygame.image.load("sun.png").convert_alpha()
sun_image = pygame.transform.scale(sun_image, (60, 60))
sun_color = (255, 200, 0)
sun_radius = 30

# Создаем все 8 планет
mercury = Planet(screen, radius=8, orbit_radius=60, color=(255, 255, 255), speed=4.1, angle=0, image_path="mercury.png", name="Mercury")
venus = Planet(screen, radius=12, orbit_radius=90, color=(255, 255, 255), speed=3.2, angle=1.5, image_path="venus.png", name="Venus")
earth = Planet(screen, radius=13, orbit_radius=120, color=(255, 255, 255), speed=2.7, angle=3.0, image_path="earth.png", name="Earth")
mars = Planet(screen, radius=10, orbit_radius=150, color=(255, 255, 255), speed=2.3, angle=4.5, image_path="mars.png", name="Mars")
jupiter = Planet(screen, radius=25, orbit_radius=200, color=(255, 255, 255), speed=1.3, angle=6.0, image_path="jupiter.png", name="Jupiter")
saturn = Planet(screen, radius=22, orbit_radius=260, color=(255, 255, 255), speed=0.9, angle=7.5, image_path="saturn.png", name="Saturn")
uranus = Planet(screen, radius=18, orbit_radius=320, color=(255, 255, 255), speed=0.7, angle=9.0, image_path="uranus.png", name="Uranus")
neptune = Planet(screen, radius=17, orbit_radius=380, color=(255, 255, 255), speed=0.5, angle=10.5, image_path="neptune.png", name="Neptune")

#создаем луну
moon = Moon(screen, radius=4, orbit_radius=20, planet=earth, color=(200, 200, 200), speed=5.0, angle=0, image_path="moon.png")

#создаем комету
comet = Comet(screen, image_path="comet.png") 

#список всех планет для удобного обновления и отрисовки
planets = [mercury, venus, earth, mars, jupiter, saturn, uranus, neptune]
total_planets = len(planets)

#переменные для счета и состояния игры
score = 0
game_won = False
scored_flag = False
font = pygame.font.Font(None, 36)
large_font = pygame.font.Font(None, 72)

#игровой цикл
running = True
while running:
    dt = clock.tick(FPS) / 1000.0
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.VIDEORESIZE:
            WIDTH, HEIGHT = event.w, event.h
            screen = pygame.display.set_mode((WIDTH, HEIGHT), pygame.RESIZABLE)
            cx = WIDTH // 2
            cy = HEIGHT // 2
        elif event.type == pygame.KEYDOWN:
            if not game_won:  #только если игра не закончена
                if event.key == pygame.K_UP:
                    angle = min(angle + 5, 80)
                elif event.key == pygame.K_DOWN:
                    angle = max(angle - 5, 10)
                elif event.key == pygame.K_w:
                    power = min(power + 1, 25)
                elif event.key == pygame.K_s:
                    power = max(power - 1, 5)
                elif event.key == pygame.K_SPACE and on_ground:  #событие броска
                    rad = math.radians(angle)
                    vx = power * math.cos(rad)
                    vy = -power * math.sin(rad)
                    on_ground = False
                    scored_flag = False

    if not game_won:
        #обновляем позиции всех планет
        for planet in planets:
            planet.update(dt)
        
        #обновляем позицию Луны
        moon.update(dt)
        
        #обновляем комету
        comet.update(dt)
        
        #обновляем позицию шарика
        if not on_ground:
            vy += gravity
            x += vx
            y += vy

        #проверка отскока от нижней границы
        if y + ball_radius >= HEIGHT:
            y = HEIGHT - ball_radius
            vy = -vy * bounce_loss
            vx *= bounce_loss
            if abs(vy) < 1 and abs(vx) < 1:
                x, y = start_x, start_y
                vx, vy = 0, 0
                on_ground = True
                scored_flag = False

        #проверяем столкновения шарика с планетами
        ball_rect = pygame.Rect(x - ball_radius, y - ball_radius, ball_radius * 2, ball_radius * 2)
        
        for planet in planets:
            if planet.active and ball_rect.colliderect(planet.rect) and not scored_flag:
                planet.hit()
                score += 1
                scored_flag = True
        
        #проверяем победу
        if score >= total_planets:
            game_won = True

    #отрисовка
    screen.fill((0, 0, 0))  
    
    #отрисовка солнца
    screen.blit(sun_image, (cx - sun_radius, cy - sun_radius))
    
    #отрисовка всех планет
    for planet in planets:
        planet.draw()
    
    #рисуем луну
    moon.draw()
    
    #отрисовка кометы
    comet.draw()
    
    #отрисовка траекторию прицелвания
    if on_ground and not game_won:
        points = []
        rad = math.radians(angle)
        temp_vx = power * math.cos(rad)
        temp_vy = -power * math.sin(rad)
        temp_x, temp_y = start_x, start_y
        for i in range(60):
            temp_vy += gravity
            temp_x += temp_vx
            temp_y += temp_vy
            if temp_y + ball_radius >= HEIGHT:
                break
            points.append((int(temp_x), int(temp_y)))
        if len(points) > 1:
            pygame.draw.lines(screen, (255, 255, 255, 100), False, points, 2)
    
    #отрисовка шарика
    pygame.draw.circle(screen, (255, 255, 255), (int(x), int(y)), ball_radius)  
    
    #счёт
    score_text = font.render(f"Сбито планет: {score}/{total_planets}", True, (255, 255, 255))
    angle_text = font.render(f"Угол: {angle}°", True, (255, 255, 255))
    power_text = font.render(f"Сила: {power}", True, (255, 255, 255))
    
    screen.blit(score_text, (10, 10))
    screen.blit(angle_text, (10, 50))
    screen.blit(power_text, (10, 90))
    
    if scored_flag and not game_won:
        hit_text = font.render("ПОПАДАНИЕ!", True, (0, 255, 0))
        screen.blit(hit_text, (250, 50))
    
    pygame.display.flip()

pygame.quit()